In [ ]:
import pandas as pd    
import numpy as np     

df_original = pd.read_csv("DAL.csv")  #Reading the dataset into a dataframe using Pandas
print(df_original.shape)               # Should give us a good idea what the dataset looks like
df_original.head()                     # looking at the top rows to make sure everything is in order

In [ ]:
# From previous we can see that there are 110 columns. Let us cut it down to only a few to be analysed
analyzed_col =['season','gameId', 'goalsFor', 'goalsAgainst', 'shotAttemptsFor', 'shotAttemptsAgainst']
# The data is split into multiple periods per game but I want to only analyse the data per game
# To accomplish that we'll use the groupby and sum functions
# Finally we want to feed it into a new data frame that we can then manipulate later
games_df = df_original[analyzed_col].groupby(['season','gameId']).sum().reset_index()   
games_df.head(10)

In [ ]:
# Checking my work by looking up these games I noticed that the goals count is doubted both For and Against
# This seems to be a problem with the data since it holds true for the 10 games I checked.
games_df['goalsFor'] = games_df['goalsFor'] /2             #A Simple solution to clean up the data
games_df['goalsAgainst'] = games_df['goalsAgainst'] /2
games_df.head(10)

In [ ]:
games_df[['goalsFor', 'goalsAgainst', 'shotAttemptsFor', 'shotAttemptsAgainst']].describe() #Get a summary of the data

In [ ]:
games_df['goalsFor'].hist()

In [ ]:
games_df['goalsAgainst'].hist() #the gap in this histogram is caused by the bin size

In [ ]:
games_df['goalsAgainst'].hist(bins=9) # The default bin size is 10, at 9 the gap dissapears

In [ ]:
#The original data did not show who won ultimately won each game
#However we have enough information to compute this ourselves

#This function takes a difference in goals and returns a string of either "Won","Lost" or "Draw"
def calc_match_result(delta_goals):
    match_result = "Won" if delta_goals > 0 else "Lost" if delta_goals < 0 else "Draw"
    return match_result


games_df['matchResult'] = (          #First we are creating an entry for each game that stores the difference in point total
    games_df['goalsFor']
    - games_df['goalsAgainst']
)
games_df['matchResult'] = [          #Then taking advantage of list comprehension we move through every entry and deploy our function
    calc_match_result(delta_goals) for delta_goals in games_df['matchResult']   
]

games_df.head()

In [ ]:
#Now that we have the match results lets look at how the team did each season

season_df = games_df[               #A newdataframe excluding gameId and grouped by season
['season', 'goalsFor', 'goalsAgainst', 'shotAttemptsFor', 'shotAttemptsAgainst', 'matchResult']
].groupby(['season']).sum().reset_index()

season_df

In [ ]:
# We don't really want the match results in that form. Instead I'd like them as a win percentage
# To get this we will need to go back to the games data frame

# This will get us the counts of all the wins, losses, and draws and organize them by season.
# Furthermore sort turned off will give up the values in the same order for every season.
# while normalizing will give us it as a percentage which will be useful for the next steps
season_results = games_df[['season','matchResult']].groupby(['season']).value_counts(sort=False,normalize=True)

season_results

In [ ]:
#We really only care about the far right values so we will separate them off into an array
results_array= season_results.array

In [ ]:
# We are once again interested mostly in the far right values

#Calculates all the indices that corresponde to "Win" and puts them into a List
def index_filler():
    for n in range(0,17):
         i=n*3+2
         results_index.append(i)

def win_percent(index):
    return results_array[index]
    
results_index=[]
index_filler()



season_df['matchResult'] = [
    win_percent(index) * 100 for index in results_index
] 
season_df.rename(columns ={'matchResult':'winPercent'}, inplace=True)
season_df